In [1]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [2]:
import pandas as pd
import numpy as np
from numpy import arange, isclose

from flatlib import const
from flatlib import aspects
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject
from flatlib.tools.chartdynamics import ChartDynamics
from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints, TransformDoubleValues, ObjectsPrepare, AspectsPrepare, AspectsClearing

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,14:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


### Get astrological objects

In [5]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [6]:
df.charts

0    <flatlib.chart.Chart object at 0x7f0dda73d730>
1    <flatlib.chart.Chart object at 0x7f0dda73d760>
2    <flatlib.chart.Chart object at 0x7f0dda73d790>
Name: charts, dtype: object

In [7]:
necessary_constants = AstrologicalConstants.necessary_constants
str(necessary_constants)

"['SUN', 'MOON', 'SATURN', 'URANUS', 'NEPTUNE', 'PLUTO', 'CHIRON', 'NORTH_NODE', 'SOUTH_NODE']"

In [8]:
rulers_constants = AstrologicalConstants.rulers_constants
str(rulers_constants)

"['ASC', 'DESC', 'MC', 'IC', 'PARS_FORTUNA']"

In [9]:
antes_objects = AstrologicalConstants.for_antes_objects
str(antes_objects)

"['moon', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'pars_fortuna', 'uranus', 'neptune', 'pluto', 'chiron']"

In [10]:
pars_constants = AstrologicalConstants.pars_constants
str(pars_constants)

"['PARS_SPIRIT', 'PARS_GLORY', 'PARS_CREST', 'PARS_ROCK']"

In [11]:
rulers_constant_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_constant_col_names

['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna']

In [12]:
rulers_pars_col_names = AstrologicalPoints.rulers_col_names(pars_constants)
rulers_pars_col_names.append('ruler_antes_pars_fortuna')
str(rulers_pars_col_names)

"['ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'ruler_antes_pars_fortuna']"

In [13]:
for obj in rulers_constants + necessary_constants + pars_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

In [14]:
for name, obj in zip(rulers_constant_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [15]:
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects(df, obj)

In [16]:
antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)      

In [17]:
pars_constants.append('ANTES_PARS_FORTUNA')

In [18]:
for name, obj in zip(rulers_pars_col_names, pars_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

In [19]:
houses_objects = AstrologicalConstants.houses_objects
houses_objects

['asc', 'desc', 'mc', 'ic']

In [20]:
str(antes_objects)

"['antes_moon', 'antes_ruler_asc', 'antes_ruler_desc', 'antes_ruler_mc', 'antes_ruler_ic', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron']"

In [21]:
pars_objects = AstrologicalConstants.pars_objects
str(pars_objects)

"['pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock']"

In [22]:
ruler_objects = AstrologicalConstants.ruler_objects
str(ruler_objects)

"['ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock']"

In [23]:
main_objects = AstrologicalConstants.main_objects
str(main_objects)

"['moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [24]:
cols_for_id = houses_objects + antes_objects + pars_objects + ruler_objects + main_objects 
print(len(cols_for_id))
str(cols_for_id)

38


"['asc', 'desc', 'mc', 'ic', 'antes_moon', 'antes_ruler_asc', 'antes_ruler_desc', 'antes_ruler_mc', 'antes_ruler_ic', 'antes_pars_fortuna', 'antes_uranus', 'antes_neptune', 'antes_pluto', 'antes_chiron', 'pars_fortuna', 'pars_spirit', 'pars_glory', 'pars_crest', 'pars_rock', 'ruler_asc', 'ruler_desc', 'ruler_mc', 'ruler_ic', 'ruler_pars_fortuna', 'ruler_antes_pars_fortuna', 'ruler_pars_spirit', 'ruler_pars_glory', 'ruler_pars_crest', 'ruler_pars_rock', 'moon', 'sun', 'saturn', 'uranus', 'neptune', 'pluto', 'chiron', 'north_node', 'south_node']"

In [25]:
df['id_for_aspects'] = AstrologicalPoints.id_for_aspekts(df, cols_for_id)

In [26]:
df['transform_id'] = df.id_for_aspects.map(lambda x: TransformDoubleValues(x).conbine_class_methods())

In [27]:
df.drop(columns='id_for_aspects', inplace=True)

In [28]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,pars_spirit,pars_glory,pars_crest,pars_rock,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_moon,antes_ruler_asc,antes_ruler_desc,antes_ruler_mc,antes_ruler_ic,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron,ruler_pars_spirit,ruler_pars_glory,ruler_pars_crest,ruler_pars_rock,ruler_antes_pars_fortuna,transform_id
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7f0dda73d730>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18 +00:00:00>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,<Pars Spirit Libra +29:20:59 +00:00:00>,<Pars Glory Sagittarius +18:23:55 +00:00:00>,<Pars Crest Sagittarius +08:56:02 +00:00:00>,<Pars Rock Aries +09:40:27 +00:00:00>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Saturn Capricorn +25:04:15 +00:00:06>,<Antes Mars Virgo +13:32:52 +00:45:19>,<Antes Venus Virgo +04:38:54 +01:12:41>,<Antes Pars Fortuna Virgo +04:31:42 +00:00:00>,<Antes Uranus Virgo +14:53:27 +00:03:14>,<Antes Neptune Libra +22:09:22 +00:02:13>,<Antes Pluto Sagittarius +14:45:25 +00:01:02>,<Antes Chiron Libra +12:23:45 +00:03:43>,<Venus Aries +25:21:06 +01:12:41>,<Jupiter Leo +13:39:12 -00:04:47>,<Jupiter Leo +13:39:12 -00:04:47>,<Mars Aries +16:27:08 +00:45:19>,<Mercury Pisces +00:37:44 +01:29:39>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_ruler_desc, Antes Saturn), (antes_ruler_mc..."
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7f0dda73d760>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29 +00:00:00>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Pars Spirit Pisces +08:49:02 +00:00:00>,<Pars Glory Capricorn +07:57:50 +00:00:00>,<Pars Crest Virgo +14:43:46 +00:00:00>,<Pars Rock Aries +08:19:08 +00:00:00>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Antes Moon Capricorn +08:48:19 +12:19:41>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Pars Fortuna Virgo +14:23:31 +00:00:00>,<Antes Uranus Virgo +07:12:20 +00:02:49>,<Antes Neptune Libra +18:05:05 +00:00:42>,<Antes Pluto Sagittarius +12:48:25 -00:00:57>,<Antes Chiron Libra +05:17:00 +00:01:47>,<Jupiter Virgo +13:33:03 +00:02:30>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Mercury Taurus +14:22:23 +00:04:22>,<Mars Sagittarius +01:25:22 -00:21:00>,<Mercury Taurus +14:22:23 +00:04:22>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_moon, Antes Moon), (antes_pars_fortuna, An..."
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7f0dda73d790>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26 +00:00:00>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:

### Calculate aspects

In [29]:
# stop

In [30]:
objects = ObjectsPrepare(df.iloc[2]).conbine_class_methods()

In [34]:
singles_degrees = AstrologicalConstants.singles_degrees
singles_degrees

[0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]

In [35]:
houses_remove_objs = []
moon_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Antes Moon','Antes Uranus', 
                      'Antes Neptune', 'Antes Pluto', 'Antes Chiron']
main_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Moon', 'Antes Moon']

In [36]:
houses_aspect_degs = {'Con': 0}
moon_aspect_degs = AstrologicalConstants.aspekts_degrees
main_aspect_degs = {'Con': 0, 'Opp': 180}

In [37]:
house_ids = ['Asc', 'Desc', 'MC', 'IC'] 

In [38]:
house_objs, moon_objs, main_objs = objects.copy(),objects.copy(), objects.copy()

asc_pluto = []
for obj in objects:
    if obj.id == 'Asc' or obj.id == 'Pluto':
        asc_pluto.append(obj)      

In [39]:
all_aspects = []

In [40]:
moon_asp_after = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs)
moon_asp_before = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs, before_point_asp='yes', before_orb=1)
all_aspects.append(moon_asp_after)
all_aspects.append(moon_asp_before)

In [41]:
for h_obj in house_ids:
    house_asp_after = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=3)
    house_asp_before = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='yes', before_orb=3)
    all_aspects.append(house_asp_after)
    all_aspects.append(house_asp_before)

In [42]:
asc_pluto_asp_after = AspectsPrepare.object_aspects('Asc', asc_pluto, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=20)

In [43]:
all_aspects.append(asc_pluto_asp_after)

In [44]:
for m_obj in objects:
    main_asp_after = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='no', after_orb=3)
    main_asp_before = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='yes', before_orb=3)
    all_aspects.append(main_asp_after)
    all_aspects.append(main_asp_before)

In [45]:
all_aspects_full = AspectsPrepare.transform_dict_list_type(all_aspects)

In [47]:
AspectsClearing.remove_duplicate_aspects(all_aspects_full)

In [49]:
AspectsClearing.remove_antes_repeat_aspects(all_aspects_full)

In [51]:
AspectsClearing.remove_duble_antes_aspects(all_aspects_full)

In [53]:
AspectsClearing.remove_node_opposition(all_aspects_full)

In [54]:
for asp in all_aspects_full:
    print(asp['f_point'], '-', asp['s_point'], '-', asp['type'])

Moon - Mercury - Sque
Moon - Antes Pars Fortuna - Sque
Pars Fortuna - Neptune - Con
Pars Fortuna - Pars Rock - Con
Uranus - South Node - Con
Neptune - Pars Rock - Con
Pars Spirit - Antes Uranus - Con
Pars Rock - Antes Mercury - Con
Mercury - Antes Pars Fortuna - Con
Jupiter - North Node - Con
Antes Jupiter - Mars - Con


In [55]:
str([x.id for x in objects])

"['Asc', 'Desc', 'MC', 'IC', 'Pars Fortuna', 'Sun', 'Moon', 'Saturn', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Pars Spirit', 'Pars Glory', 'Pars Crest', 'Pars Rock', 'Mercury', 'Jupiter', 'Antes Moon', 'Antes Mercury', 'Antes Jupiter', 'Antes Pars Fortuna', 'Antes Uranus', 'Antes Neptune', 'Antes Pluto', 'Antes Chiron', 'Mars', 'Venus']"

In [56]:
# for x in objects:
#     if x.id == 'Mars':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 2      

In [57]:
str(['conv_compl', 'conv_denide']) # for septener planet and parses

"['conv_compl', 'conv_denide']"

In [58]:
# Rules for aspekts:
# Antes Moon - only house conuction
# Remove:
# Doublecation similar planets and antes aspects  
# Moon Node opp
# Add:
# Will exist aspect function for rulers Asc and Dsc - 
# - denide of aspect from aspect with other planet (create after getting all point aspects)

In [59]:
chart = df.charts.values[0]
chart

In [60]:
asc = chart.get(const.ASC)
moon = chart.get(const.MOON)
sun = chart.get(const.SUN)

In [61]:
from flatlib.object import Object

In [62]:
sun = chart.get(const.SUN)

In [63]:
# antes_pars_fortuna_obj 

In [64]:
sun.lon

352.66547108970656

In [65]:
asc_obj = Object.fromDict({'id': asc.id, 'sign': asc.sign, 'signlon': asc.signlon, 'lonspeed': 0, 'orb': asc.orb})

In [66]:
moon_obj = Object.fromDict({'id': moon.id, 'sign': moon.sign, 'signlon': moon.signlon, 'lonspeed': moon.lonspeed, 'orb': moon.orb})

In [67]:
antes_pars_fortuna_obj = Object.fromDict({'id': 'Antes Pars Fortuna'})#, 'sign': sun.sign, 'signlon': sun.signlon, 'lonspeed': sun.lonspeed})#, 'orb': sun.orb})

In [68]:
antes_pars_fortuna_obj.orb()

3

In [69]:
moon_asc_aspect = aspects.getAspect(moon_obj, asc_obj, const.MAJOR_ASPECTS)
print('moon_asc_aspect: ', moon_asc_aspect)

moon_asc_aspect:  <Moon Asc 0 Exact +00:00:00>


In [70]:
dyn = ChartDynamics(chart)

In [71]:
moon_asps = dyn.immediateAspects('North Node', const.MAIN_ASPECTS)
print(moon_asps)  

(None, None)


In [72]:
from flatlib.lists import GenericList

In [73]:
list_obj = [antes_pars_fortuna_obj]

In [74]:
gen_list = GenericList(list_obj)
gen_list

In [75]:
gen_list.add = sun_obj

NameError: name 'sun_obj' is not defined

In [ ]:
gen_list.content

In [ ]:
chart.get('Antes Pars Fortuna')

In [ ]:
chart.objects.add(antes_pars_fortuna_obj)

In [ ]:
chr_obj = [x for x in chart.objects]

In [ ]:
len (chr_obj)

In [ ]:
chr_obj[15].id

In [ ]:
dyn_obj = ChartDynamics(chart)

In [ ]:
moon_asps = dyn_obj.immediateAspects('Moon', const.MAJOR_ASPECTS)
moon_asps

In [ ]:
chart.getObject(ID='Moon')

In [ ]:
df.antes_moon[0]

In [ ]:
# Create new columns for each object, five clasess for favorite's advantsge and orbs's 